In [1]:
import os 
print(os.getcwd())

/home/liuchen/jupyterprj/transferlearning


In [2]:
####处理数据
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './re/train', 
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=32,
        class_mode='categorical')                               # matt，多分类

validation_generator = test_datagen.flow_from_directory(
        './re/test',
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')                             # matt，多分类
# class_mode='binary'

Using TensorFlow backend.


Found 400 images belonging to 5 classes.
Found 100 images belonging to 5 classes.


In [3]:
#迁移学习

from keras.applications.inception_v3 import InceptionV3, preprocess_input
base_model = InceptionV3(weights='imagenet', include_top=False)

In [4]:
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model

def add_new_last_layer(base_model, nb_classes):
#       """Add last layer to the convnet
#       Args:
#         base_model: keras model excluding top
#         nb_classes: # of classes
#       Returns:
#         new keras model with last layer
#       """
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(FC_SIZE, activation='relu')(x) #new FC layer, random init
    predictions = Dense(nb_classes, activation='softmax')(x) #new softmax layer
    model = Model(input=base_model.input, output=predictions)
    return model


print(type(base_model))
nb_classes=5
FC_SIZE = 1024
model = add_new_last_layer(base_model, nb_classes)

<class 'keras.engine.training.Model'>
Instructions for updating:
keep_dims is deprecated, use keepdims instead


/home/liuchen/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  app.launch_new_instance()


In [5]:
def setup_to_transfer_learn(model, base_model):
#     """Freeze all layers and compile the model"""
    print(type(model))
    for layer in base_model.layers:
        layer.trainable = False
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

setup_to_transfer_learn(model=model,base_model=base_model)

<class 'keras.engine.training.Model'>
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [6]:
 history_tl = model.fit_generator(
    train_generator,
    nb_epoch=10,
    samples_per_epoch=500,
    validation_data=validation_generator,
    nb_val_samples=200,
    class_weight='auto')

/home/liuchen/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  import sys
/home/liuchen/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., class_weight="auto", steps_per_epoch=15, epochs=10, validation_steps=200)`
  import sys


Epoch 1/10
15/15 [==============================] - 233s - loss: 3.3456 - acc: 0.4933 - val_loss: 4.9558 - val_acc: 0.5100
Epoch 2/10
15/15 [==============================] - 229s - loss: 0.4001 - acc: 0.8852 - val_loss: 0.1316 - val_acc: 0.9698
Epoch 3/10
15/15 [==============================] - 230s - loss: 0.4270 - acc: 0.8629 - val_loss: 4.1781 - val_acc: 0.5000
Epoch 4/10
15/15 [==============================] - 229s - loss: 0.2598 - acc: 0.9374 - val_loss: 0.0937 - val_acc: 0.9702
Epoch 5/10
15/15 [==============================] - 231s - loss: 0.3396 - acc: 0.9065 - val_loss: 1.9400 - val_acc: 0.8500
Epoch 6/10
15/15 [==============================] - 230s - loss: 0.3497 - acc: 0.9209 - val_loss: 0.0568 - val_acc: 0.9900
Epoch 7/10
15/15 [==============================] - 229s - loss: 0.1703 - acc: 0.9732 - val_loss: 0.3567 - val_acc: 0.9700
Epoch 8/10
15/15 [==============================] - 230s - loss: 0.0937 - acc: 0.9770 - val_loss: 0.7309 - val_acc: 0.8506
Epoch 9/10
15/15

In [9]:
###预测
from keras.preprocessing import image
from PIL import Image
import numpy as np


print(os.getcwd())
img='300.jpg'
img=Image.open(img)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = model.predict(x)
print(preds)

/home/liuchen/jupyterprj/transferlearning
[[0.02023092 0.08731461 0.83536935 0.04406026 0.01302483]]
